# **HMM Viterbi, Forward and Backward**
# Importing the required Libraries

In [29]:
import numpy as np
import matplotlib.pyplot as plt

# Defining the functions for Veterbi Algorithm
## It performs the veterbi algorithm and return the Best state sequence and the probabbility for that sequence.

In [30]:
def veterbi(states,o,i,t,seq):
  state_seq = [i]
  p = i
  prob = [1]
  for j in range(len(seq)):
    for k in range(len(state_seq)):
      s1 = state_seq[k]
      s2 = prob[k]
      p = state_seq[k][len(state_seq[k])-1]
      for l in range(len(states)):
        if l == 0:
          state_seq[k] = state_seq[k] + states[l]
          prob[k] = prob[k] * t[(p,seq[j],states[l])]
        else: 
          new_seq = s1 + states[1]
          new_p = s2 * t[(p,seq[j],states[1])]  
          state_seq.append(new_seq)
          prob.append(new_p)
  argmax = prob.index(max(prob))
  return state_seq[argmax],prob[argmax]

# Defining the Function for Alpha and Beta Algorithm

In [31]:
def alpha(states,o,i,t,seq):
  alpha = []
  a = []
  for j in range(len(states)):
    if states[j] == i:
      a.append(1)
    else:
      a.append(0)
  alpha.append(a)
  for j in range(len(seq)):
    a = alpha[j]
    n = []
    for k in range(len(states)):
      sum = 0
      for l in range(len(states)):
        sum+=a[l]*t[(states[l],seq[j],states[k])]
      n.append(sum)
    alpha.append(n)
  return alpha

def beta(states,o,i,t,seq):
  beta = []
  b = []
  for j in range(len(states)):
    b.append(1)
  beta.append(b)
  j = len(seq)-1
  while j != -1:
    b = beta[len(seq)-1 - j]
    n = []
    for k in range(len(states)):
      sum = 0
      for l in range(len(states)):
        sum+=b[l]*t[(states[k],seq[j],states[l])]
      n.append(sum)
    if j == 0:
      for m in range(len(states)):
        if states[m] != i:
          n[m] = 0
    beta.append(n)
    j = j - 1 
  return beta

def get_probability(x):
  p = []
  for j in range(len(x)):
    sum = 0
    for k in range(len(x[j])):
      sum += x[j][k]
    p.append(sum)
  return p 

def get_probability_state(a,b,p):
  s = []
  for i in range(len(a)):
    l =[]
    for j in range(len(a[i])):
      l.append((a[i][j]*b[i][j])/p[i])
    s.append(l)
  return s

In [32]:
def updated_tranitions(states,o,i,t,seq):
  a = alpha(states,o,i,t,seq)
  b1 = beta(states,o,i,t,seq)
  b = []
  for j in range(len(b1)):
    b.append(b1[len(b1)-1-j])
  p = get_probability(a)
  c = t.copy()
  d = []
  #print(a)
  #print(b)
  for j in range(len(states)):
    for k in range(len(o)):
      for l in range(len(states)):
        sum = 0
        pb = 1
        for m in range(len(seq)):
          pb = pb * t[(states[j],o[k],states[l])]
          sum += a[m][j]*pb*b[m+1][l]
          #print(sum," ",a[m][j]*pb[m][j]*b[m+1][l],a[m][j],pb[m][j],b[m+1][l],"\n")
        #print(sum)
        #sum = sum / p
        c[(states[j],o[k],states[l])]=sum
  #print(c)
  for j in range(len(states)):
    sum = 0
    for k in range(len(o)):
      for l in range(len(states)):
        sum = sum + c[(states[j],o[k],states[l])]
    d.append(sum)
  #print(d)
  for j in range(len(states)):
    for k in range(len(o)):
      for l in range(len(states)):
        if d[j]!=0:
          #print(c[(states[j],o[k],states[l])]/d[j])
          c[(states[j],o[k],states[l])] = c[(states[j],o[k],states[l])]/d[j]
  return c

def get_transition_prob(s,o,a,b):
  new_t = {}
  for i in range(len(s)):
    for j in range(len(o)):
      for k in range(len(s)):
        new_t[(s[i],o[j],s[k])] = a[(s[i],s[k])]*b[(o[j],s[i])]
  return new_t

def difference(states,o,x,y):
  d = []
  for j in range(len(states)):
    for k in range(len(o)):
      for l in range(len(states)):
        d.append(x[(states[j],o[k],states[l])]-y[(states[j],o[k],states[l])])
  return d 

def hmm_training(states,o,i,t,seq):
  z = 0
  initial = t.copy()
  update = updated_tranitions(states,o,i,t,seq).copy()
  #print(update)
  for j in range(15):
    update_next = updated_tranitions(states,o,i,update,seq).copy()
    diff = difference(states,o,update_next,update)
    #print(update_next)
    update = update_next.copy()
    if np.sum(diff) == 0:
      z = z + 1
      if z == 2:
        break
  return update 

# Taking the input HMM

In [33]:
states = input("Enter comma seprated States:\n").split(",")
i = input("Enter initial state:\n")
observable = input("Enter comma seprated variables:\n").split(",")
t = {}
for k in range(len(states)):
  for l in range(len(observable)):
    for m in range(len(states)):
      s = "Enter the probability for "+states[k]+" -> "+states[m]+" for the observation "+observable[l]+"\n"
      t[states[k],observable[l],states[m]] = float(input(s))
print(t)

Enter comma seprated States:
q,r,s
Enter initial state:
q
Enter comma seprated variables:
a,b
Enter the probability for q -> q for the observation a
0
Enter the probability for q -> r for the observation a
0.25
Enter the probability for q -> s for the observation a
0.25
Enter the probability for q -> q for the observation b
0
Enter the probability for q -> r for the observation b
0.25
Enter the probability for q -> s for the observation b
0.25
Enter the probability for r -> q for the observation a
0.5
Enter the probability for r -> r for the observation a
0
Enter the probability for r -> s for the observation a
0
Enter the probability for r -> q for the observation b
0.5
Enter the probability for r -> r for the observation b
0
Enter the probability for r -> s for the observation b
0
Enter the probability for s -> q for the observation a
0.5
Enter the probability for s -> r for the observation a
0
Enter the probability for s -> s for the observation a
0
Enter the probability for s -> q 

In [34]:
sequence = input("Enter the input Sequence:\n")
state_s,pro = veterbi(states,observable,i,t,sequence)
print("Best State Sequence for",sequence,"is",state_s)
print("Best State Sequence Probability for",sequence,"is",pro)

Enter the input Sequence:
aabb
Best State Sequence for aabb is qrqrq
Best State Sequence Probability for aabb is 0.015625


In [35]:
a = alpha(states,observable,i,t,sequence)
p = get_probability(a)
print("The Alpha vector for time tick are:",a)
print("The value of p1,w is:",p[len(p)-1])

The Alpha vector for time tick are: [[1, 0, 0], [0.0, 0.25, 0.25], [0.25, 0.0, 0.0], [0.0, 0.0625, 0.0625], [0.0625, 0.0, 0.0]]
The value of p1,w is: 0.0625


In [36]:
b = beta(states,observable,i,t,sequence)
p = get_probability(b)
print("The Beta Vectors for time Tick are:",b)
print("The probility of p1,w is:",p[len(p)-1])

The Beta Vectors for time Tick are: [[1, 1, 1], [0.5, 0.5, 0.5], [0.25, 0.25, 0.25], [0.125, 0.125, 0.125], [0.0625, 0, 0]]
The probility of p1,w is: 0.0625


In [37]:
result = hmm_training(states,observable,i,t,sequence)
#print(get_transition_prob(states,observable,A,B))
#print(t)
print(result)

{('q', 'a', 'q'): 0.0, ('q', 'a', 'r'): 0.25, ('q', 'a', 's'): 0.25, ('q', 'b', 'q'): 0.0, ('q', 'b', 'r'): 0.25, ('q', 'b', 's'): 0.25, ('r', 'a', 'q'): 0.5, ('r', 'a', 'r'): 0.0, ('r', 'a', 's'): 0.0, ('r', 'b', 'q'): 0.5, ('r', 'b', 'r'): 0.0, ('r', 'b', 's'): 0.0, ('s', 'a', 'q'): 0.5, ('s', 'a', 'r'): 0.0, ('s', 'a', 's'): 0.0, ('s', 'b', 'q'): 0.5, ('s', 'b', 'r'): 0.0, ('s', 'b', 's'): 0.0}
